In [1]:
#import
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

In [2]:
#Reading dataset
df=pd.read_csv('Morgan_desc.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 2049 entries, Yield to 1023.1
dtypes: int64(2049)
memory usage: 1.1 MB


In [3]:
#Preparing

# delete variables with zero variance
Var0Variable = np.where( df.var(axis=0) == 0 )
if len(Var0Variable[0]) == 0:
    print( "There is no variable with zero variance." )
    print( "" )
else:
    print( "There were {0} variables with zero variance".format(len(Var0Variable[0])))
    print( "the number of variable is: {0}".format(Var0Variable[0]) )
    print( "remove them" )
    print( "" )
    #df_var0 = np.delete(df, Var0Variable, 1)    
    df_var0 = df.drop(df.columns[Var0Variable], axis=1)

#removal of columns which data has all 0
def remove_all_zero_col(df):
    df = df.copy()
    for col in df.columns:
        if (df[col] == 0).all():
            df.drop(col, axis=1, inplace=True)
    return df
df_var0 = remove_all_zero_col(df_var0)
df_var0.info()

There were 1914 variables with zero variance
the number of variable is: [   1    2    3 ... 2046 2047 2048]
remove them

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 135 entries, Yield to 919.1
dtypes: int64(135)
memory usage: 71.8 KB


In [4]:
#Building descriptors
X = df_var0.drop(columns=['Yield'])

print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df_var0['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   8  9  15  33  58  63  64  68  80  102  ...  623.1  665.1  695.1  726.1  \
0  0  0   0   0   0   0   1   0   0    0  ...      1      1      1      0   
1  0  0   0   0   0   0   1   0   0    0  ...      1      0      0      0   
2  0  0   0   0   0   0   1   0   0    0  ...      1      0      0      1   
3  0  0   0   0   0   0   1   0   0    0  ...      0      0      0      0   
4  0  0   0   1   0   0   0   0   0    0  ...      1      1      1      0   

   755.1  802.1  807.1  841.1  849.1  919.1  
0      0      1      0      1      0      0  
1      1      0      1      0      0      0  
2      0      0      1      0      1      1  
3      0      0      1      0      0      0  
4      0      1      0      1      0      0  

[5 rows x 134 columns]
---Objective---
   Yield
0     80
1      7
2      3
3     48
4     99


In [6]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []

#knn
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_train=a_X_train.dropna(axis=1)
    a_X_test=a_X_test.dropna(axis=1)
    
    param_knn={'n_neighbors':[1,2,3,4,5,6,7,8,9,10], 'p':[1,2,3,4,5]}
    reg_knn = GridSearchCV(KNeighborsRegressor(), param_grid=param_knn, cv=5, n_jobs=2)
    reg_knn.fit(a_X_train,y_train['Yield'])
    reg_best = reg_knn.best_estimator_
    y_pred1 = reg_best.predict(a_X_train)
    y_pred2 = reg_best.predict(a_X_test)

    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)

    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    
    parametors = reg_knn.best_params_
    best_model_parametors.append(parametors)
    cv_score = reg_knn.best_score_
    data_cv_score.append(cv_score)
 
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_knn.best_params_)
    print("Best Model Score:",reg_knn.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'n_neighbors': 7, 'p': 1}
Best Model Score: 0.6671568457745215
R2_test: 0.5053476756978456
----------------------
seed: 1
Best Model Parameter: {'n_neighbors': 8, 'p': 1}
Best Model Score: 0.581446065463125
R2_test: 0.6126121823377695
----------------------
seed: 2
Best Model Parameter: {'n_neighbors': 8, 'p': 1}
Best Model Score: 0.6084680981687987
R2_test: 0.6234657256362639
----------------------
seed: 3
Best Model Parameter: {'n_neighbors': 9, 'p': 1}
Best Model Score: 0.4896050327873859
R2_test: 0.6136458302705987
----------------------
seed: 4
Best Model Parameter: {'n_neighbors': 9, 'p': 1}
Best Model Score: 0.5158453549987059
R2_test: 0.6192411943383311
----------------------
seed: 5
Best Model Parameter: {'n_neighbors': 9, 'p': 1}
Best Model Score: 0.5955125394863127
R2_test: 0.5278183472222395
----------------------
seed: 6
Best Model Parameter: {'n_neighbors': 8, 'p': 1}
Best Model Score: 0.5945247668027119
R2_test: 0.652

In [7]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['MSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])
        
data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd], 
                     axis=1, join='inner')
data_all.loc['mean'] = data_all.mean()
print(data_all)

data_all.to_csv('../../score/MorganF/knn_score.csv')

      r2_train   MSE_train  MAE_train   r2_test   RMSE_test   MAE_test  \
0     0.714181  210.724707  11.723404  0.505348  646.980564  19.870748   
1     0.642883  322.944481  13.976064  0.612612  375.319940  14.678571   
2     0.695046  282.112699  13.162234  0.623466  364.131696  16.684524   
3     0.646895  329.344366  14.028369  0.613646  370.801881  15.925926   
4     0.630472  318.568689  13.756501  0.619241  404.870076  15.714286   
5     0.681264  258.092987  12.160757  0.527818  574.269253  17.904762   
6     0.693151  251.822806  12.763298  0.652426  418.290923  13.779762   
7     0.683528  302.704733  13.826748  0.531658  424.555879  17.768707   
8     0.763851  207.838763  10.917553  0.620572  374.964286  15.988095   
9     0.676160  328.193816  14.523936  0.680281  225.292411  13.041667   
mean  0.682743  281.234805  13.083886  0.598707  417.947691  16.135705   

      cv_score  
0     0.667157  
1     0.581446  
2     0.608468  
3     0.489605  
4     0.515845  
5     0.5